In [2]:
import numpy as np
import pandas as pd

In [4]:
#Stats
df= pd.read_csv('/home/gibeon/Desktop/iHub/Data/Train.csv')

#df['Pos'] = df.index
#df.reset_index(level=0, inplace=True)

#df['INDX'] = pd.Series(list(range(len(df['Team']))))
df.iloc[:10, 1:25]

User Id Vehicle Type  Platform Type Personal or Business  \
0   User_Id_633         Bike              3             Business   
1  User_Id_2285         Bike              3             Personal   
2   User_Id_265         Bike              3             Business   
3  User_Id_1402         Bike              3             Business   
4  User_Id_1737         Bike              1             Personal   
5  User_Id_1342         Bike              3             Business   
6  User_Id_2803         Bike              3             Business   
7  User_Id_1075         Bike              3             Business   
8   User_Id_733         Bike              3             Business   
9  User_Id_2112         Bike              3             Business   

   Placement - Day of Month  Placement - Weekday (Mo = 1) Placement - Time  \
0                         9                             5       9:35:46 AM   
1                        12                             5      11:16:16 AM   
2                        30                             2      12:39:25 PM   
3                        15                             5       9:25:34 AM   
4                        13                             1       9:55:18 AM   
5                        14                             5       3:07:35 PM   
6                         9                             5       9:33:45 AM   
7                        11                             1       2:13:01 PM   
8                        30                             2      11:10:44 AM   
9                        23                             5       4:48:54 PM   

   Confirmation - Day of Month  Confirmation - Weekday (Mo = 1)  \
0                            9                                5   
1                           12                                5   
2                           30                                2   
3                           15                                5   
4                           13                                1   
5                           14                                5   
6                            9                                5   
7                           11                                1   
8                           30                                2   
9                           23                                5   

  Confirmation - Time  ...  Pickup - Weekday (Mo = 1)  Pickup - Time  \
0          9:40:10 AM  ...                          5    10:27:30 AM   
1         11:23:21 AM  ...                          5    11:44:09 AM   
2         12:42:44 PM  ...                          2    12:53:03 PM   
3          9:26:05 AM  ...                          5     9:43:06 AM   
4          9:56:18 AM  ...                          1    10:05:23 AM   
5          3:08:57 PM  ...                          5     3:30:30 PM   
6          9:49:47 AM  ...                          5     9:56:45 AM   
7          2:14:13 PM  ...                          1     2:22:40 PM   
8         11:15:49 AM  ...                          2    12:22:57 PM   
9          5:17:56 PM  ...                          5     5:34:38 PM   

  Arrival at Destination - Day of Month  \
0                                     9   
1                                    12   
2                                    30   
3                                    15   
4                                    13   
5                                    14   
6                                     9   
7                                    11   
8                                    30   
9                                    23   

   Arrival at Destination - Weekday (Mo = 1)  Arrival at Destination - Time  \
0                                          5                    10:39:55 AM   
1                                          5                    12:17:22 PM   
2                                          2                     1:00:38 PM   
3                                          5                    10:05:27 AM   


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV 
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

import xgboost as xgb
import lightgbm as lgb

sns.set(style='white', context='notebook', palette='deep')

In [5]:
#!conda install -c conda-forge lightgbm -y

Solving environment: done

## Package Plan ##

  environment location: /home/gibeon/anaconda3

  added / updated specs:
    - lightgbm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    lightgbm-2.3.0             |   py37he1b5a44_0         1.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  lightgbm           conda-forge/linux-64::lightgbm-2.3.0-py37he1b5a44_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda                                            anaconda --> conda-forge



conda-4.7.12         | 3.0 MB    | ##################################### | 100% 
lightgbm-2.3.0       | 1.0 MB    | ##################################### | 100

In [6]:
#Train_Masked has extra columns: Delivery destination (day, month, time)

train_df = pd.read_csv("/home/gibeon/Desktop/iHub/Data/Train.csv")
test_df = pd.read_csv("/home/gibeon/Desktop/iHub/Data/Test.csv")
riders_df = pd.read_csv("/home/gibeon/Desktop/iHub/Data/Riders.csv")

print(train_df.shape, test_df.shape, riders_df.shape)
train_df.head()

(21201, 29) (7068, 25) (960, 5)


Order No       User Id Vehicle Type  Platform Type  \
0   Order_No_4211   User_Id_633         Bike              3   
1  Order_No_25375  User_Id_2285         Bike              3   
2   Order_No_1899   User_Id_265         Bike              3   
3   Order_No_9336  User_Id_1402         Bike              3   
4  Order_No_27883  User_Id_1737         Bike              1   

  Personal or Business  Placement - Day of Month  \
0             Business                         9   
1             Personal                        12   
2             Business                        30   
3             Business                        15   
4             Personal                        13   

   Placement - Weekday (Mo = 1) Placement - Time  Confirmation - Day of Month  \
0                             5       9:35:46 AM                            9   
1                             5      11:16:16 AM                           12   
2                             2      12:39:25 PM                           30   
3                             5       9:25:34 AM                           15   
4                             1       9:55:18 AM                           13   

   Confirmation - Weekday (Mo = 1)  ... Arrival at Destination - Time  \
0                                5  ...                   10:39:55 AM   
1                                5  ...                   12:17:22 PM   
2                                2  ...                    1:00:38 PM   
3                                5  ...                   10:05:27 AM   
4                                1  ...                   10:25:37 AM   

   Distance (KM)  Temperature Precipitation in millimeters  Pickup Lat  \
0              4         20.4                          NaN   -1.317755   
1             16         26.4                          NaN   -1.351453   
2              3          NaN                          NaN   -1.308284   
3              9         19.2                          NaN   -1.281301   
4              9         15.4                          NaN   -1.266597   

   Pickup Long Destination Lat  Destination Long      Rider Id  \
0    36.830370       -1.300406         36.829741  Rider_Id_432   
1    36.899315       -1.295004         36.814358  Rider_Id_856   
2    36.843419       -1.300921         36.828195  Rider_Id_155   
3    36.832396       -1.257147         36.795063  Rider_Id_855   
4    36.792118       -1.295041         36.809817  Rider_Id_770   

  Time from Pickup to Arrival  
0                         745  
1                        1993  
2                         455  
3                        1341  
4                        1214  

[5 rows x 29 columns]

In [7]:
for col in train_df.columns:
    if col not in test_df.columns:
        print(col)

Arrival at Destination - Day of Month
Arrival at Destination - Weekday (Mo = 1)
Arrival at Destination - Time
Time from Pickup to Arrival


In [8]:
#Checking how our target variable was calculated

labels = pd.to_datetime(train_df['Arrival at Destination - Time']) - pd.to_datetime(train_df['Pickup - Time'])
list(labels.dt.seconds[:5]), list(train_df['Time from Pickup to Arrival'][:5])

([745, 1993, 455, 1341, 1214], [745, 1993, 455, 1341, 1214])